Código em jupyter para poder facilitar a implementação e atualização do meu projeto pessoal de atualização automática de planilha de controle financeiro. Especificamente a planilha de cartões. Dessa vez tentar implementar a automação também do cartão de crédito da caixa.

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd 
from time import sleep
from pprint import pprint
import credenciais
from datetime import datetime, date
from google.oauth2 import service_account
from googleapiclient.discovery import build



In [26]:
options = webdriver.ChromeOptions()
options.add_argument('--log-level=3')
url = 'https://www2.bancobrasil.com.br/aapf/login.html?1638208647003#/acesso-aapf-agencia-conta-1'

chrome = webdriver.Chrome(
    options=options,
    service=ChromeService(ChromeDriverManager().install())
    )


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 12.4MB/s]


In [27]:
chrome.get(url)
agencia = credenciais.agencia_bb
conta = credenciais.conta_bb
senha = credenciais.senha_bb

In [28]:
# Preencher o campo agência
chrome.find_element(By.ID,"dependenciaOrigem").send_keys(agencia)
# Preencher o campo conta e apertar enter
chrome.find_element(
    By.ID,"numeroContratoOrigem"
    ).send_keys(conta, Keys.ENTER)

In [29]:
# Preencher o campo senha e apertar Enter
chrome.find_element(By.ID,"senhaConta").send_keys(senha, Keys.ENTER)

In [30]:
# Digita Cartões no campo de busca e aperta Enter
chrome.find_element(By.ID,"acheFacil").send_keys("cartões", Keys.ENTER)
sleep(4)
# Escolhe a opção de extrato e entra
act = ActionChains(chrome)
act.click(chrome.find_element(
    By.XPATH,"//a[contains(text(),'Cartões - Fatura - Extrato')]"
    )).perform()

In [31]:
# Seleciona o Cartão Elo Nanquim
chrome.find_element(By.XPATH,'//*[@id="carousel1"]/div/div/img[2]').click()

In [32]:
# Extrair a data de fechamento da próxima fatura
chrome.find_element(By.XPATH,'//*[@id="faturasAtual"]/li[11]/a').click()
sleep(2)
elemento = chrome.find_element(
    By.XPATH,'//*[@id="fatura2"]/table/tbody/tr[4]/td/div/ul/li/span'
    )
html_content = elemento.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')
fechamento_fatura_elo = soup.get_text('\n')
print(fechamento_fatura_elo)

23/03/2023


In [33]:
# Clica em fatura atual para ver os histórico de compras recente
chrome.find_element(By.XPATH,"//*[@id='faturasAtual']/li[12]").click()

In [34]:
# Rola a página para baixo
chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [35]:
# Captura a tabela de informações presente na página
elo_nanquim = []
try:
    elemento = chrome.find_element(
        By.XPATH,"//*[@id='fatura2']/div[9]/table")
    html_content = elemento.get_attribute('outerHTML')
    # Método organiza as informações em html
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')
    # Método que extrai as informações do html e coloca num data frame
    extrato_full = pd.read_html(str(table))[0]
    extrato = extrato_full[[0, 1, 2]]
    extrato.columns = ['Data', 'Descrição', 'Valor']
    extrato = extrato.drop([0, 1], axis=0)
    # Extrai os dados do data frame e manda para um dicionário
    historico_compras = {}
    historico_compras['extrato'] = extrato.to_dict('records')

    lista = []
    elo_nanquim = []
    for linha in historico_compras['extrato']:
        lista.append(linha['Data'])
        lista.append(linha['Descrição'])
        lista.append(int(linha['Valor'])/100)
        elo_nanquim.append(lista)
        lista = []
except Exception as err:
    print(err)
pprint(elo_nanquim)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='fatura2']/div[9]/table"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00276643]
	(No symbol) [0x0020BE21]
	(No symbol) [0x0010DA9D]
	(No symbol) [0x00141342]
	(No symbol) [0x0014147B]
	(No symbol) [0x00178DC2]
	(No symbol) [0x0015FDC4]
	(No symbol) [0x00176B09]
	(No symbol) [0x0015FB76]
	(No symbol) [0x001349C1]
	(No symbol) [0x00135E5D]
	GetHandleVerifier [0x004EA142+2497106]
	GetHandleVerifier [0x005185D3+2686691]
	GetHandleVerifier [0x0051BB9C+2700460]
	GetHandleVerifier [0x00323B10+635936]
	(No symbol) [0x00214A1F]
	(No symbol) [0x0021A418]
	(No symbol) [0x0021A505]
	(No symbol) [0x0022508B]
	BaseThreadInitThunk [0x76CB7D69+25]
	RtlInitializeExceptionChain [0x7747BB9B+107]
	RtlClearBits [0x7747BB1F+191]
	(No symbol) [0x00000000]

[]


In [36]:
# Selecionar o cartão Smiles Visa Infinite
chrome.find_element(By.XPATH,'//*[@id="carousel1"]/div/div/img[3]').click()

In [37]:
# Clica em fatura atual para ver os histórico de compras recente
chrome.find_element(By.CLASS_NAME,"inactive").click()

In [38]:
# Rolar a página para baixo para ter acesso aos gastos recentes
chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [39]:
# Extrair tabela com os gastos recentes
visa = []
try:
    elemento = chrome.find_element(
        By.XPATH,"//*[@id='fatura2']/div[9]//table")
    html_content = elemento.get_attribute('outerHTML')

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')

    extrato_full = pd.read_html(str(table))[0]
    extrato = extrato_full[[0, 1, 2]]
    extrato.columns = ['Data', 'Descrição', 'Valor']
    extrato = extrato.drop([0, 1], axis=0)

    historico_compras = {}
    historico_compras['extrato'] = extrato.to_dict('records')

    lista = []
    visa = []
    for linha in historico_compras['extrato']:
        lista.append(linha['Data'])
        lista.append(linha['Descrição'])
        lista.append(int(linha['Valor'])/100)
        visa.append(lista)
        lista = []
except Exception as err:
    print(err)
pprint(visa)
chrome.close()

invalid literal for int() with base 10: '16.687,00'
[['24/02', 'ROSA GOMES DE SOU', 10.0],
 ['24/02', 'CASARAO DA VILA R', 71.23],
 ['24/02', 'AUTO POSTO BRAGA LTDA', 70.0],
 ['24/02', 'COCO BAMBU AGUAS', 84.15],
 ['24/02', 'DF PLAZA LTDA', 15.7],
 ['25/02', 'ATACADAO DROGACEN', 74.97]]


In [40]:
# Concatenar as tabelas de gastos dos dois cartões
lista_definitiva = visa + elo_nanquim
pprint(lista_definitiva)

[['24/02', 'ROSA GOMES DE SOU', 10.0],
 ['24/02', 'CASARAO DA VILA R', 71.23],
 ['24/02', 'AUTO POSTO BRAGA LTDA', 70.0],
 ['24/02', 'COCO BAMBU AGUAS', 84.15],
 ['24/02', 'DF PLAZA LTDA', 15.7],
 ['25/02', 'ATACADAO DROGACEN', 74.97]]


In [41]:
fechamento_fatura = fechamento_fatura_elo
dicionario_meses = {1: "janeiro", 2: "fevereiro", 3: "março",
                    4: "abril", 5: "maio", 6: "junho",
                    7: "julho", 8: "agosto", 9: "setembro",
                    10: "outubro", 11: "novembro", 12: "dezembro"}
aba_mes = int(fechamento_fatura.split('/')[1]) + 1
if aba_mes == 13:
    aba_mes = 1


In [42]:
# Setar o range da aba mês api!C2 - Fechamento da Fatura
range_aba = f'{dicionario_meses[aba_mes]}!F11'

# Pegar data atual para decidir qual mês irei adicionar os dados
data_atual = date.today()
data_atual = f'{data_atual.day}/{data_atual.month}/{data_atual.year}'

In [43]:
print(f'Fechamento da próxima fatura Visa: {fechamento_fatura}')
print(f'Histórico de compras Visa: {lista_definitiva}')

Fechamento da próxima fatura Visa: 23/03/2023
Histórico de compras Visa: [['24/02', 'ROSA GOMES DE SOU', 10.0], ['24/02', 'CASARAO DA VILA R', 71.23], ['24/02', 'AUTO POSTO BRAGA LTDA', 70.0], ['24/02', 'COCO BAMBU AGUAS', 84.15], ['24/02', 'DF PLAZA LTDA', 15.7], ['25/02', 'ATACADAO DROGACEN', 74.97]]


In [45]:
SERVICE_ACCOUNT_FILE = 'keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [46]:
# The ID spreadsheet.
SAMPLE_SPREADSHEET_ID = '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI'
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [48]:
# Inserir data de fechamento da fatura visa
sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                    range=range_aba,
                    valueInputOption="USER_ENTERED",
                    body={"values": [[fechamento_fatura]]}).execute()

{'spreadsheetId': '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI',
 'updatedRange': 'Abril!F11',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [49]:
# Update da Sheets
descricao_compra = []
for gasto in lista_definitiva:
    preço = str(f'{gasto[2]:.2f}').split('.')
    gasto[2] = "R$ " + preço[0] + "," + preço[1]
    descricao_compra.append(gasto)
pprint(descricao_compra)

[['24/02', 'ROSA GOMES DE SOU', 'R$ 10,00'],
 ['24/02', 'CASARAO DA VILA R', 'R$ 71,23'],
 ['24/02', 'AUTO POSTO BRAGA LTDA', 'R$ 70,00'],
 ['24/02', 'COCO BAMBU AGUAS', 'R$ 84,15'],
 ['24/02', 'DF PLAZA LTDA', 'R$ 15,70'],
 ['25/02', 'ATACADAO DROGACEN', 'R$ 74,97']]


In [52]:
# O número da aba será coletado de acordo com a data do gasto
ano = int(data_atual.split('/')[2])
range = f'{dicionario_meses[aba_mes]}!H2:J100'
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range).execute().get('values', [])
pprint(result)

[['09/11', 'DROGARIA ROSARIO 5/5', 'R$ 181,04'],
 ['28/11', 'Suporte Notebook 4/4', 'R$ 27,78'],
 ['29/11', 'Fone de ouvido 4/4', 'R$ 35,87'],
 ['12/11', 'Tapete dos meninos 5/5', 'R$ 37,98'],
 ['19/12', 'Passagem Gol 4/5 ', 'R$ 340,42'],
 ['02/01', 'Fraldas Heitor 3/3', 'R$ 73,35'],
 ['05/01', 'Roupão Renner 3/3', 'R$ 103,20'],
 ['06/01', 'Ultimas Faculdade 3/3', 'R$ 73,40'],
 ['06/01', 'Ultimas Faculdade 3/3', 'R$ 74,39'],
 ['13/01', 'Marcação de assentos 3/4', 'R$ 65,00'],
 ['16/01', 'Manual 3/3', 'R$ 161,34'],
 ['16/01', 'Globo Bateria 3/4', 'R$ 97,50'],
 ['18/01', 'Aluguel do Carro 3/6', 'R$ 135,78'],
 ['01/02', 'Ritalina 2/4', 'R$ 58,22'],
 ['05/02', 'Livros Rafael 2/10', 'R$ 102,00'],
 ['06/02', 'Clube Smiles', 'R$ 39,90']]


In [ ]:
planilha = []
for row in result:
    data = row[0].split('/')
    row[0] = data[0] + "/" + data[1]
    planilha.append(row)
pprint(planilha)

In [55]:
# Comparando os gastos com os valores já na planilha
resultado_compra = []
for i in descricao_compra:
    if i not in planilha:
        resultado_compra.append(i)

In [57]:
# Pegar a data de fechamento da fatura na planilha e alimentá-la de acordo
fechamento_planilha = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range_aba).execute().get('values', [])[0][0]
print(fechamento_planilha)

23/03/2023


In [58]:
import time
# Transformar data de fechamento da planilha em timestamp
timestamp_fechamento = time.mktime(datetime.strptime(
    fechamento_planilha, '%d/%m/%Y').timetuple())
print(timestamp_fechamento)

1679540400.0


In [ ]:
# Enviar mensagem via telegram caso exista gasto a ser adicionado
# Alimentar a planilha com os dados do cartão Visa
validador_msg = 0
for gasto in resultado_compra:
    numero_aba = int(gasto[0].split('/')[1]) + 1
    if numero_aba == 13:
            numero_aba = 1
    # if mes_gasto > numero_aba:
    #     data_gasto = f'{gasto[0]}/{ano-1}'
    
    data_gasto = f'{gasto[0]}/{ano}'

    timestamp_data_gasto = time.mktime(datetime.strptime(
                                data_gasto, '%d/%m/%Y').timetuple())

    if timestamp_data_gasto < timestamp_fechamento:
        # Mandar msg para cada gasto
        mensagem_gasto = f'{gasto[1]} \- {gasto[-1]}'
        if "=" in mensagem_gasto:
            mensagem_gasto = mensagem_gasto.replace('=', '\=')
        if "*" in mensagem_gasto:
            mensagem_gasto = mensagem_gasto.replace('*', '\*')
        # telegram_bot(mensagem_gasto)

        validador_msg = 1
        range = f'{dicionario_meses[numero_aba]}!H2:J100'
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range).execute().get('values', [])
        next_empty_row = len(result) + 2
        range = f'{dicionario_meses[numero_aba]}!H{next_empty_row}'
        sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                    range=range,
                    valueInputOption="USER_ENTERED",
                    body={"values": [gasto]}).execute()
    else:
        numero_aba = numero_aba + 1
        if numero_aba == 13:
            numero_aba = 1
        
        range = f'{dicionario_meses[numero_aba]}!H2:J100'
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range).execute().get('values', [])
        
        planilha = []
        for row in result:
            data = row[0].split('/')
            row[0] = data[0] + "/" + data[1]
            planilha.append(row)

        if gasto not in planilha:
            mensagem_gasto = f'{gasto[1]} \- {gasto[-1]}'
            if "=" in mensagem_gasto:
                mensagem_gasto = mensagem_gasto.replace('=', '\=')
            if "*" in mensagem_gasto:
                mensagem_gasto = mensagem_gasto.replace('*', '\*')
            # telegram_bot(mensagem_gasto)

            validador_msg = 1
            next_empty_row = len(result) + 2
            range = f'{dicionario_meses[numero_aba]}!H{next_empty_row}'
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range,
                        valueInputOption="USER_ENTERED",
                        body={"values": [gasto]}).execute()